## 关联分析
从大规模数据集中寻找物品间的隐含关系被称作关联分析（association analysis）或者关联规则学习（association rule learning）。

这些关系可以有两种形式：频繁项集或者关联规则。频繁项集（frequent item sets）是经常出现在一块的物品的集合，关联规则（association rules）暗示两种物品之间可能存在很强的关系。

支持度和可信度是被用来量化关联分析是否成功的方法。

支持度：一个项集的支持度（support）被定义为数据集中包含该项集的记录所占的比例。支持度是针对项集来说的，因此可以定义一个最小支持度，而只保留满足最小支持度的项集。

可信度或置信度（confidence）是针对一条诸如{A}->{B}的关联规则来定义的。这条规则的可信度被定义为"支持度({A,B})/支持度({A})"

关联分析的目标：发现频繁项集和发现关联规则。首先需要找到频繁项集，然后才能获得关联规则。


### Apriori原理
当有很多元素时，便会产生很多组合，为了计算每种组合的支持度以及可信度我们需要长时间的运算。为了降低运算时间，可以采用Apriori原理。

Apriori原理:如果某个项集是频繁的，那么它的所有子集都是频繁的。

直观上看这个原理并无帮助，但是反过来看，一个项集是非频繁集，那么它的所有超集也是非频繁的。

Apriori算法是发现频繁项集的一种方法。输入最小支持度与数据集。算法先生成所有单个物品的项集列表。接着扫描交易记录来查看哪些项集满足最小支持度要求，那些不满足最小支持度的集合将被去掉。然后，对剩下的集合进行组合以生成包含两个元素的项集。接下来在重新扫描交易记录，去掉不满足最小支持度的项集。如此重复直至所有项集都被去掉。

In [18]:
#辅助函数

#数据集扫描伪代码
#对数据集中的每条交易记录tran
#对每个候选项集can：
#    检查一下can是否是tran的子集：
#    如果是，则增加can的计数值
#对每个候选项集：
#如果其支持度不低于最小值，则保留该项集
#返回所有频繁项集列表

from numpy import *

def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

#构建集合C1，C1是大小为1的所有候选项集的集合
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:#遍历记录
        for item in transaction:#遍历记录中的每一项
            if not [item] in C1:#如果该项并未出现在C1
                C1.append([item])#这里是添加了项的一个列表 即意味着C1是一个集合的集合，如{{1},{2}...}
    C1.sort()
    return list(map(frozenset,C1)) #对C1中每个项构建一个不变集合  frozenset是python的一种数据类型，它们是不可变的

#数据集扫描
def scanD(D, Ck, minSupport):#数据集、包含候选集合的列表、最小支持度
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(list(D)))                 
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems #计算所有项集的支持度
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData #返回频繁项集列表和频繁项集的支持度

dataSet=loadDataSet()
C1=createC1(dataSet)
D = [set(var) for var in dataSet]
print(D)
L1,suppData0=scanD(D,C1,0.5)
L1

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]


[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [23]:
#完整Apriori算法
#伪代码
#当集合中项的个数大于0时
#    构建一个k个项组成的候选项集的列表
#    检查数据以确认每个项集都是频繁的
#    保留频繁项集并构建k+1项组成的候选项集的列表

def aprioriGen(Lk, k): #频繁项集列表、项集元素个数
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #如果前k-2个项相同时，将两个集合合并
                retList.append(Lk[i] | Lk[j]) 
    return retList#输出候选项集

def apriori(dataSet, minSupport = 0.5):#数据集合、支持度
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#扫描数据集 从Ck得到Lk 丢掉不满足最小支持度的项集
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

#创建6个不重复的两元素集合
L,supportData=apriori(dataSet)
#print(L)#L包含满足最小支持度为0.5的频繁项集列表 每个项集都是函数apriori调用aprioriGen()来的

#aprioriGen函数工作过程
print(L[0])
aprioriGen(L[0],2)

#在70%支持度的输出
#L,supportData=apriori(dataSet,minSupport=0.7)
#L

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]


[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

**通过上述函数，我们可以发现频繁项集，现在需要从其中找出关联规则**

In [26]:
#关联规则生成函数

#主函数
def generateRules(L, supportData, minConf=0.7):  #频繁项集列表、包含那些频繁项集支持数据的字典、最小可信度阈值
    bigRuleList = []
    for i in range(1, len(L)):#只获取有两个或更多元素的集合
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):#如果频繁项集元素数目大于2，则通过一下函数对其进行合并
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:#不然则计算可信度值
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList #包含可信度的规则列表    

#生成候选规则集合
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] 
    for conseq in H:#变量H中所有项集并计算器可信度
        conf = supportData[freqSet]/supportData[freqSet-conseq] #计算可信度
        if conf >= minConf: #如果某条规则满足最小可信度值，那么输出该规则
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH #返回一个满足最小可信度要求的规则列表

#对规则进行评估
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):#频繁项集、可以出现在规则右部的元素列表H
    m = len(H[0])#H中频繁项集列表大小
    if (len(freqSet) > (m + 1)): #检查频繁项集是否大到可以移除大小为m的子集
        Hmp1 = aprioriGen(H, m+1)#生成H中元素的无重复组合 即创建Hm+1条新候选规则
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)#Hmp1包含所有可能的规则。利用calcConf()来测试它们的可信度以确定规则是否满足要求
        if (len(Hmp1) > 1):    #如果不止一条规则满足要求
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)#继续调用，判断是否可以进一步组合这些规则
            
L,supportData=apriori(dataSet,minSupport=0.5)
rules=generateRules(L,supportData,minConf=0.7)
#rules=generateRules(L,supportData,minConf=0.5)降低可信度阈值会获得更多的规则
rules

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

### 总结
关联分析是用于发现大数据集中元素关系的一个工具集，可以采用两种方式来量化这些有趣的关系：
* 使用频繁项集，它会给出经常在一起出现的元素项
* 关联规则，每条关联规则意味着元素项之间的“如果......那么”关系

Apriori算法优缺点：
* 优点：易编码实现
* 缺点：在大数据集上可能较慢
* 适用数据类型：数值型或者标称型数据

因为每次增加频繁项集的大小，Apriori算法都会重新扫描整个数据集。当数据集很大时，就会比较慢。在此引入FPgrowth算法，该算法只需要对数据库进行两次遍历，能够显著加快发现频繁项集的速度。

### FP-growth算法
它基于Apriori构建，但采用了一些不同的技术。将数据集存储在一个特定的称作FP树的结构之后发现频繁项集或频繁项对，即常在一块出现的元素项的集合FP树

FP-growth算法会扫描数据集两次，它发现频繁项集的基本过程如下：
1. 构建FP树
2. 从FP树中挖掘频繁项集

FP（Frequent Pattern）代表频繁模式。FP树通过链接（link）来连接相似元素。

关于FP树：https://www.cnblogs.com/zhengxingpeng/p/6679280.html

FP-growth算法对原始数据扫描两次：第一遍用来统计出现的频率，第二遍扫描中只考虑那些频繁元素。

In [2]:
class treeNode:
    #存放节点名字的变量和1个计数值
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None #nodeLink用于链接相似的元素
        self.parent = parentNode      #指向父节点
        self.children = {} 
    
    def inc(self, numOccur):#对count变量增加给定量
        self.count += numOccur
        
    def disp(self, ind=1):#将树以文本形式显示
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)
            
rootNode=treeNode('pyramid',9,None)
rootNode.children['eye']=treeNode('eye',13,None)
rootNode.children['phoenix']=treeNode('phoenix',3,None)
rootNode.disp()

   pyramid   9
     eye   13
     phoenix   3


In [4]:

def createTree(dataSet, minSup=1): #使用数据集记最小支持度作为参数构建FP树
    headerTable = {}
    #遍历两次数据集
    for trans in dataSet:#扫描数据集，并统计各个元素项出现的频度
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable.keys()):  #移除不满足最小支持度的元素项
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print('freqItemSet: ',freqItemSet)
    if len(freqItemSet) == 0: return None, None  #如果没有元素项满足，则退出
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #扩展头指针表以便报错计数值及指向每种类型第一个元素项的指针
    #print('headerTable: ',headerTable)
    retTree = treeNode('Null Set', 1, None) #建树
    for tranSet, count in dataSet.items():  #再一次遍历数据集，这次只考虑那些频繁项
        localD = {}
        for item in tranSet:  #根据全局频率对每个事务中的元素进行排序
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#使用排序后的频繁项集对树进行填充
    return retTree, headerTable #返回树和头指针表

#用于让树生长，输入参数为一个项集
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#首先测试事务中的第一个元素项是否作为子节点存在
        inTree.children[items[0]].inc(count) #如果存在，则更新该元素项的计数
    else:   #不存在则创建一个新的treenode并将其作为子节点添加到树中
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #更新头指针表
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#不断迭代调用自身，每次调用会去掉列表的第一个元素
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

#用于确保节点链接指向树中该元素项的每一个实例
def updateHeader(nodeToTest, targetNode):   
    while (nodeToTest.nodeLink != None):    #不要使用递归遍历链表，因为如果链表很长会遇到迭代调用的次数限制
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode
    
    
    
#数据集与建树
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

simpDat=loadSimpDat()
initSet=createInitSet(simpDat)
myFPtree,myHeaderTab=createTree(initSet,3)
myFPtree.disp()

   Null Set   1
     z   5
       r   1
       x   3
         s   2
           y   2
             t   2
         r   1
           y   1
             t   1
     x   1
       r   1
         s   1


构建好树之后就是抽取频繁项集，步骤如下：
1. 从FP树中获得条件模式基
2. 利用条件模式基，构建一个条件FP树
3. 迭代重复步骤（1）步骤（2），直到树包含一个元素项为止

条件模式基（conditional pattern base）是以所查找元素项为结尾的路径集合。每一条路径其实都是一条前缀路径（prefix path）。一条前缀路径是介于所查找元素项与树根节点之间的所有内容。

In [5]:
#发现以给定元素项结尾的所有路径的函数

def ascendTree(leafNode, prefixPath): #迭代上溯整棵树
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
    
def findPrefixPath(basePat, treeNode): #遍历链表知直到到达结尾
    condPats = {}
    while treeNode != None: #每遇到一个元素就调用ascendTree()来上溯FP树，并收集所有遇到的元素项的名称
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats #条件模式基字典


findPrefixPath('x',myHeaderTab['x'][1])
#findPrefixPath('x',myHeaderTab['z'][1])
#findPrefixPath('x',myHeaderTab['r'][1])

{frozenset({'z'}): 3}

可以发现上述的返回值与表中结果一致。有了条件模式基之后，就可以创建条件FP树。对于每个频繁项集，都要创建一棵条件FP树。

In [8]:
#递归查找频繁项集的函数
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]#对头指针表中的元素项按照其出现频率进行排序（默认从小到大）
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print('finalFrequent Item: ',newFreqSet)   #append to set
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])#  #递归调用创建条件基
        #print('condPattBases :',basePat, condPattBases)
        myCondTree, myHead = createTree(condPattBases, minSup)#该条件基被作为新数据传递给createTree函数 从条件模式基来构建条件FP树
        #print('head from conditional tree: ', myHead)
        if myHead != None: #挖掘条件FP树
            #print('conditional tree for: ',newFreqSet)
            #myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList) #如果树中有元素项的话，递归调用mineTree()

freqItems=[]
mineTree(myFPtree,myHeaderTab,3,set([]),freqItems)
freqItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'y'},
 {'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'t'},
 {'t', 'y'},
 {'t', 'z'},
 {'t', 'y', 'z'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]

### 总结
FP-growth优缺点
* 优点：一般要快于Apriori
* 缺点：实现比较困难，在某些数据集上性能会下降
* 适用数据类型：标称型数据